# In questo notebook conto quante volte un utente ha utilizzato un determinato hashtag

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import csv

In [3]:
spark = SparkSession.builder.appName("hashtag analysis").config("spark.executor.memory", "500m").getOrCreate()

In [4]:
df = spark.read.options(sep=",", header=True, quote="\"", escape="\"", multiline=True).csv('covidusersandhashtags.csv')

In [5]:
tuples_RDD = df.rdd

In [6]:
user_list_RDD = tuples_RDD.filter(f=lambda e: not(isinstance(e[0], type(None))) and not(isinstance(e[1], type(None))))

In [7]:
user_word_RDD = user_list_RDD.flatMap(f=lambda l: [(l[0], hashtag) for hashtag in (l[1]).split(" ")])

In [8]:
userandword_to_one_RDD = user_word_RDD.map(f=lambda t: (t,1))

In [9]:
userandword_to_sum_RDD = userandword_to_one_RDD.reduceByKey(func=lambda a,b: a+b)

In [10]:
without_spaces_RDD = userandword_to_sum_RDD.filter(f=lambda e: not(e[0][1]==""))

In [11]:
result_RDD = without_spaces_RDD.map(f=lambda x: x[0][0] + "," + x[0][1] + "," + str(x[1]))

In [12]:
result_RDD.saveAsTextFile('Conteggio_hashtag_covidusers')